# 🔍 DoH Watcher
DNS over HTTPS (DoH) の効果を可視化・比較する教育向けツール。

このノートブックでは、Google DoH と Cloudflare DoH を使って DNS クエリを実行し、応答時間を比較します。プライバシーとセキュリティの観点から、通常のDNSとの違いを体験的に学びましょう。


In [ ]:
# ドメインリストとクエリ回数の設定
domains = ['example.com', 'google.com', 'cloudflare.com']
query_count = 5


In [ ]:
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
DOH_GOOGLE = "https://dns.google/resolve"
DOH_CLOUDFLARE = "https://cloudflare-dns.com/dns-query"

def resolve_doh(domain, provider='google'):
    try:
        if provider == 'google':
            start = time.time()
            r = requests.get(DOH_GOOGLE, params={'name': domain, 'type': 'A'})
            elapsed = (time.time() - start) * 1000
        elif provider == 'cloudflare':
            headers = {'accept': 'application/dns-json'}
            start = time.time()
            r = requests.get(DOH_CLOUDFLARE, params={'name': domain, 'type': 'A'}, headers=headers)
            elapsed = (time.time() - start) * 1000
        if r.status_code == 200:
            return elapsed
        else:
            return None
    except Exception:
        return None


In [ ]:
results = []

for domain in domains:
    for i in range(query_count):
        elapsed_g = resolve_doh(domain, provider='google')
        elapsed_c = resolve_doh(domain, provider='cloudflare')
        results.append({
            'domain': domain,
            'provider': 'Google DoH',
            'elapsed_ms': elapsed_g
        })
        results.append({
            'domain': domain,
            'provider': 'Cloudflare DoH',
            'elapsed_ms': elapsed_c
        })

df = pd.DataFrame(results)
df.head()


In [ ]:
avg_df = df.groupby(['domain', 'provider'])['elapsed_ms'].mean().reset_index()

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(data=avg_df, x='domain', y='elapsed_ms', hue='provider')
plt.title("DoH DNS Query Response Time Comparison")
plt.ylabel("Average Response Time (ms)")
plt.xlabel("Domain")
plt.tight_layout()
plt.show()


## 🧠 学びのポイント

- 通常のDNSは暗号化されていないため、第三者から中身が見えてしまう。
- DoH（DNS over HTTPS）は、HTTPSトンネル内にDNSクエリを通すことで、プライバシーとセキュリティを向上させます。
- 一方で、通信の遅延や企業内監視の困難さといったトレードオフも存在します。
- 教育・評価目的として、DoHは重要な現代インターネットの学習対象です。
